# Clip 5

## Initial

In [1]:
import open3d as o3d
import numpy as np
import json

In [2]:
# Initial
kota = "../data/kota_circuit2.ply"
pcd = o3d.io.read_point_cloud(kota)
print(pcd)
print()

PointCloud with 14640197 points.



In [3]:
# Visualize pcd
o3d.visualization.draw_geometries([pcd])

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


KeyboardInterrupt: 

In [3]:
# Configuration for cropping

min_bound = [-150, -100, -100]
max_bound = [444, 350, 0]

print(f"min_bound = {min_bound}")
print(f"max_bound = {max_bound}")

min_bound = [-150, -100, -100]
max_bound = [444, 350, 0]


In [4]:
# Define the axis-aligned bounding box with the loaded bounds
aabb = o3d.geometry.AxisAlignedBoundingBox(min_bound=min_bound, max_bound=max_bound)

# Crop the point cloud
cropped_pcd = pcd.crop(aabb)

print(cropped_pcd)

PointCloud with 10275773 points.


## Get vertex normal

In [5]:
# Get vertex normal
cropped_pcd.estimate_normals(
    search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
print("Finish estimate normal")

Finish estimate normal


In [7]:
o3d.io.write_point_cloud("cropped_normal.ply", cropped_pcd, write_ascii=True) # save new ascii ply

True

In [12]:
# Visualize cropped
o3d.visualization.draw_geometries([cropped_pcd])

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


KeyboardInterrupt: 

## Normal Filter

In [6]:
# Prepare data for filter

# Convert the point cloud to a NumPy array
points = np.asarray(cropped_pcd.points)
colors = np.asarray(cropped_pcd.colors)
normals = np.asarray(cropped_pcd.normals)

# Print the min and max of each coordinate (test)
print(f"X range: {points[:, 0].min()} to {points[:, 0].max()}")
print(f"Y range: {points[:, 1].min()} to {points[:, 1].max()}")
print(f"Z range: {points[:, 2].min()} to {points[:, 2].max()}")

X range: -149.99990844726562 to 260.0782165527344
Y range: -99.99994659423828 to 349.9996337890625
Z range: -99.99986267089844 to -16.549089431762695


In [7]:
# Define a function to calculate the angle with the vertical
def angle_with_vertical(normal):
    # Assuming the vertical is along the z-axis
    vertical = np.array([0, 0, 1])
    cosine_angle = np.dot(normal, vertical) / (np.linalg.norm(normal) * np.linalg.norm(vertical))
    return np.arccos(cosine_angle)  # Return the angle in radians

In [10]:
# Filter out points based on the angle with the vertical
angle_threshold = np.pi / 6  # 30 degrees, adjust this threshold as needed
filtered_points = []
filtered_normals = []
filtered_colors = []

for point, normal, color in zip(cropped_pcd.points, cropped_pcd.normals, cropped_pcd.colors):
    if angle_with_vertical(normal) < angle_threshold:
        filtered_points.append(point)
        filtered_normals.append(normal)
        filtered_colors.append(color)

# Create a new point cloud with the filtered points
road_pcd = o3d.geometry.PointCloud()
road_pcd.points = o3d.utility.Vector3dVector(filtered_points)
road_pcd.normals = o3d.utility.Vector3dVector(filtered_normals)
road_pcd.colors = o3d.utility.Vector3dVector(filtered_colors)

print("Finish Calculating")

Finish Calculating


In [11]:
# Save the filtered point cloud
o3d.io.write_point_cloud("filtered_road.ply", road_pcd, write_ascii=True)

True

In [12]:
# Visualize result
o3d.visualization.draw_geometries([road_pcd])

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


KeyboardInterrupt: 